In [1]:
import warnings
warnings.filterwarnings('ignore')

In [2]:
#!/usr/bin/python3
# -*- coding: UTF-8 -*-
import time

import os
import numpy as np
# import keras
from keras import metrics
from keras.utils import to_categorical
from keras.models import load_model
from keras.preprocessing.text import Tokenizer
from keras.preprocessing.sequence import pad_sequences
from keras.models import Sequential
from keras.layers import Dense, Dropout, Flatten, SimpleRNN, GRU,Activation
from keras.layers.embeddings import Embedding
from datetime import datetime
from gensim.models import word2vec
import pandas as pd

Using TensorFlow backend.


In [73]:
# get texts data
category2idx = {'Japan_Travel': 0, 'KR_ENTERTAIN': 1, 'Makeup': 2, 'Tech_Job':  3, 'WomenTalk': 4,
                  'babymother': 5, 'e-shopping': 6, 'graduate': 7, 'joke': 8, 'movie': 9}

train_df_sample = pd.read_pickle('train.pkl').sample(frac=1, random_state=123)
# train_df_sample = pd.concat([train_pickle_df.text]).sample(frac=1)

In [125]:
# get word embedding vector
answer = word2vec.Word2Vec.load("word2vec_20180424.model")
# print(type(answer))
word_vectors = answer.wv
wvv = word_vectors.vocab
wvv_keys = wvv.keys()
wvv_keys_list = list(wvv_keys)
# print(wvv_keys_list[:10]) #['櫻花林', '好比', '考科', '床上', '一點現', '記住', '寶寶的', '柔嫩', '不規則', '朴智妍']

In [97]:
train_texts = train_df_sample.values
label_list = to_categorical(train_df_sample.category)

test_pickle_df = pd.read_pickle('test.pkl')
test_texts = test_pickle_df["text"].values

In [135]:
def text_to_index(corpus):
    new_corpus = []
    for doc in corpus:
        new_doc = []
        for word in doc:
            try:
                new_doc.append(word_index[word])
            except:
                new_doc.append(0)
#         new_doc_arr = np.array(new_doc).reshape(1, max_doc_word_length)
        new_doc_arr = np.array(new_doc)
        new_corpus.append( new_doc_arr)
    return np.array(new_corpus)

In [99]:
train_texts_list = []
for text in train_texts:
    train_texts_list.append(text[0])

# train_texts_index = train_texts_list

print(len(train_texts_list), train_texts_list[0])

9000 ['挑色', '挑', '尺寸', '挑', '款式', '挑', '尺寸', '人', '資料', '簡述', '胸無肉', '最近', '黛妮', '蠶絲', '衣尺', '碼為', '32D', '上圍', '84', '下圍', '6566', '這幾件', '內衣', '尺寸', '謎樣', '實在', '不想', '買', '再', '轉讓', '想', '說', '最近', '收到', '這件', '內衣', '請', '幫幫', '挑選', '挑選', '需求', '能夠', '集中', '不調', '皮亂', '跑', 'spanclasshl', '選手', '號', 'span', '物品', '連結', 'ahrefhttpgooglMHr6zrelnofollowtargetblankhttpgooglMHr6za', '集中', '款猶豫', '不決', '32C', '32Dspanclasshl', '選手', '號', 'span', '物品', '連結', 'ahrefhttpgooglPbBOvrelnofollowtargetblankhttpgooglPbBOva', '深', 'V', '款猶豫', '不決', '32C', '32B']


In [100]:
# train_texts_list1 = list(train_texts_list)
# print(type(train_texts_list), len(train_texts_list), type(train_texts_list[0]))
# print(train_texts_list[0])

In [101]:
tokenizer = Tokenizer()
tokenizer.fit_on_texts(train_texts_list)
sequences = tokenizer.texts_to_sequences(train_texts_list)
# max_doc_word_length = max(len(l) for l in train_texts)
max_doc_word_length = 200
sequences1 = pad_sequences(sequences, maxlen=max_doc_word_length, padding='post')
word_index = tokenizer.word_index
print("Found %s unique tokens" % len(word_index))


# data = pad_sequences(sequences)
# print("Shape of data tensor:" , data.shape)

Found 232236 unique tokens


In [127]:
vocab_size = len(word_index) + 1
# create a weight matrix for words in training docs
answer_vector_size = answer.vector_size
embedding_matrix = np.zeros((vocab_size, answer_vector_size))
for word, i in word_index.items():
    if word in wvv_keys_list:
        embedding_vector = answer[word]
        if embedding_vector is not None:
            embedding_matrix[i+1] = embedding_vector
#     else:
#         embedding_matrix[i] = np.zeros((1, answer_vector_size))

In [103]:
# del answer

In [128]:
embedding_layer = Embedding(input_dim= embedding_matrix.shape[0],
                            output_dim= embedding_matrix.shape[1],
                            weights=[embedding_matrix], 
                            trainable=False)
model = Sequential()
model.add(embedding_layer)
model.add(GRU(16))    
model.add(Dense(100, activation='relu'))
model.add(Dense(100, activation='relu'))
model.add(Dense(10, activation='softmax'))
# early_stopping = keras.callbacks.EarlyStopping(monitor='val_loss', min_delta=0.00001, patience=0, verbose=0, mode='auto')
model.compile(optimizer='adam',loss='categorical_crossentropy',  metrics=["accuracy"])
model.summary()

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding_4 (Embedding)      (None, None, 250)         58059250  
_________________________________________________________________
gru_4 (GRU)                  (None, 16)                12816     
_________________________________________________________________
dense_10 (Dense)             (None, 100)               1700      
_________________________________________________________________
dense_11 (Dense)             (None, 100)               10100     
_________________________________________________________________
dense_12 (Dense)             (None, 10)                1010      
Total params: 58,084,876
Trainable params: 25,626
Non-trainable params: 58,059,250
_________________________________________________________________


In [136]:
X_train_texts = text_to_index(train_df_sample.text)
X_train = pad_sequences(X_train_texts, maxlen= max_doc_word_length)

In [137]:
# X_train_texts_ = list(X_train_texts)
print(len(X_train), type(X_train))
print(len(X_train[0]), type(X_train[0]))
print(X_train[0].shape)
print(X_train[1].shape)

9000 <class 'numpy.ndarray'>
200 <class 'numpy.ndarray'>
(200,)
(200,)


In [138]:
history = model.fit(x = X_train, y = label_list, 
                    validation_split=0.1, 
                    batch_size= 3000,
                    epochs = 100, verbose = 1)

Train on 8100 samples, validate on 900 samples
Epoch 1/100
8100/8100 [==============================] - 18s 2ms/step - loss: 2.3020 - acc: 0.1106 - val_loss: 2.2934 - val_acc: 0.1144
Epoch 2/100
8100/8100 [==============================] - 11s 1ms/step - loss: 2.2863 - acc: 0.1369 - val_loss: 2.2810 - val_acc: 0.1244
Epoch 3/100
8100/8100 [==============================] - 10s 1ms/step - loss: 2.2722 - acc: 0.1522 - val_loss: 2.2680 - val_acc: 0.1356
Epoch 4/100
8100/8100 [==============================] - 10s 1ms/step - loss: 2.2575 - acc: 0.1542 - val_loss: 2.2541 - val_acc: 0.1522
Epoch 5/100
8100/8100 [==============================] - 10s 1ms/step - loss: 2.2416 - acc: 0.1691 - val_loss: 2.2401 - val_acc: 0.1600
Epoch 6/100
8100/8100 [==============================] - 11s 1ms/step - loss: 2.2259 - acc: 0.1801 - val_loss: 2.2263 - val_acc: 0.1656
Epoch 7/100
8100/8100 [==============================] - 11s 1ms/step - loss: 2.2111 - acc: 0.1914 - val_loss: 2.2132 - val_acc: 0.1811
E

8100/8100 [==============================] - 10s 1ms/step - loss: 1.1709 - acc: 0.5988 - val_loss: 1.3478 - val_acc: 0.5333
Epoch 61/100
8100/8100 [==============================] - 10s 1ms/step - loss: 1.1571 - acc: 0.6006 - val_loss: 1.3288 - val_acc: 0.5500
Epoch 62/100
8100/8100 [==============================] - 10s 1ms/step - loss: 1.1412 - acc: 0.6052 - val_loss: 1.3052 - val_acc: 0.5578
Epoch 63/100
8100/8100 [==============================] - 11s 1ms/step - loss: 1.1262 - acc: 0.6084 - val_loss: 1.2933 - val_acc: 0.5611
Epoch 64/100
8100/8100 [==============================] - 10s 1ms/step - loss: 1.1141 - acc: 0.6128 - val_loss: 1.2862 - val_acc: 0.5689
Epoch 65/100
8100/8100 [==============================] - 10s 1ms/step - loss: 1.1013 - acc: 0.6177 - val_loss: 1.2819 - val_acc: 0.5733
Epoch 66/100
8100/8100 [==============================] - 10s 1ms/step - loss: 1.0875 - acc: 0.6230 - val_loss: 1.2674 - val_acc: 0.5667
Epoch 67/100
8100/8100 [==============================

In [139]:
# loss_func_name = 'categorical_crossentropy'
# for value in history.history["categorical_crossentropy"]:
#     print(history)
model.save('my_model.h5') 

In [140]:
# evaluate the model
loss_accuracy = model.evaluate(sequences1[0:100], label_list[0:100], verbose=1)
print(type(loss_accuracy), loss_accuracy)

100/100 [==============================] - 0s 2ms/step
<class 'list'> [2.1207345390319823, 0.42]


In [146]:
# test_tokenizer = Tokenizer()
# test_tokenizer.fit_on_texts(test_texts)
# test_sequences = test_tokenizer.texts_to_sequences(test_texts)
# test_sequences1 = pad_sequences(test_sequences, maxlen=max_doc_word_length, padding='post')

Y_sequences1 = text_to_index(test_pickle_df.text)
Y_sequences11 = pad_sequences(Y_sequences1, maxlen=max_doc_word_length)

In [148]:
predict_res = model.predict(Y_sequences11, verbose=1)
# print(len(predict_res), predict_res)

final_res = []
for pre_res in predict_res:
    final_res.append(np.argmax(pre_res))
print(final_res)

1000/1000 [==============================] - 1s 1ms/step
[4, 9, 4, 5, 8, 8, 8, 1, 0, 0, 5, 6, 6, 8, 7, 8, 9, 3, 3, 8, 5, 3, 9, 2, 7, 6, 8, 6, 8, 4, 0, 0, 4, 9, 3, 7, 5, 6, 5, 7, 6, 9, 3, 5, 8, 9, 6, 0, 8, 2, 5, 9, 7, 3, 6, 7, 5, 9, 3, 5, 5, 5, 0, 5, 1, 5, 9, 9, 0, 0, 3, 5, 0, 7, 8, 7, 1, 4, 4, 2, 6, 4, 1, 4, 4, 2, 2, 5, 2, 0, 4, 3, 5, 1, 9, 3, 6, 2, 4, 4, 7, 9, 2, 5, 3, 7, 3, 6, 5, 3, 1, 8, 8, 1, 4, 2, 9, 5, 7, 5, 8, 1, 5, 2, 8, 3, 1, 7, 5, 1, 4, 2, 4, 7, 8, 8, 5, 5, 1, 0, 5, 7, 4, 3, 2, 2, 2, 3, 0, 0, 7, 3, 9, 2, 4, 8, 0, 1, 2, 1, 2, 2, 8, 1, 2, 3, 8, 4, 6, 7, 7, 4, 1, 3, 3, 4, 2, 7, 9, 9, 1, 3, 8, 2, 2, 7, 4, 7, 0, 0, 3, 7, 1, 5, 5, 9, 8, 3, 4, 5, 7, 7, 2, 8, 1, 7, 9, 3, 3, 9, 8, 2, 4, 6, 3, 4, 6, 7, 2, 6, 0, 6, 2, 8, 9, 3, 8, 2, 9, 9, 1, 3, 5, 3, 4, 6, 9, 4, 3, 2, 9, 9, 6, 2, 6, 1, 6, 7, 8, 3, 5, 7, 0, 6, 1, 5, 4, 5, 4, 3, 7, 8, 1, 1, 2, 2, 2, 0, 7, 3, 7, 9, 6, 8, 4, 1, 7, 9, 9, 0, 1, 5, 8, 7, 3, 5, 5, 1, 2, 6, 1, 0, 9, 7, 9, 3, 6, 1, 5, 1, 0, 7, 0, 7, 5, 4, 7, 5, 7, 0, 6, 8, 6, 9, 

In [149]:
# result_txt = "result" + str(datetime.now()).split()[1] + ".txt"
print(len(final_res))
result_txt = "result001" + ".txt"
ids = 0
with open(result_txt, 'w') as out:
    out.write("id,category" + '\n')
    for value in final_res:
        out.write(str(ids) + "," + str(value) + '\n')
        ids += 1

1000


In [19]:
# summ = 0
# for s in test_texts[100:110]:
# #     summ += len(s)
#     print(s)
# # print(summ)